<h2>【0.821】TFIDF_LR_simple_baseline</h2>

Data from [all-in-one-dataset](https://www.kaggle.com/adldotori/all-in-one-dataset/notebook)

# Import Library

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import re 
import scipy
from scipy import sparse

from IPython.display import display
from pprint import pprint
from matplotlib import pyplot as plt 

import time
import scipy.optimize as optimize
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_colwidth=300
pd.options.display.max_columns = 100

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from nltk.tokenize import word_tokenize
from datasets import Dataset

# Prepare train data

In [ ]:
import pandas as pd
import numpy as np
import copy
import re
from keras.preprocessing.text import text_to_word_sequence
from nltk import WordNetLemmatizer


class BaseTokenizer(object):
    def process_text(self, text):
        raise NotImplemented

    def process(self, texts):
        for text in texts:
            yield self.process_text(text)


RE_PATTERNS = {
    ' american ':
        [
            'amerikan'
        ],
    ' adolf ':
        [
            'adolf'
        ],
    ' hitler ':
        [
            'hitler'
        ],
    ' fuck':
        [
            '(f)(u|[^a-z0-9 ])(c|[^a-z0-9 ])(k|[^a-z0-9 ])([^ ])*',
            '(f)([^a-z]*)(u)([^a-z]*)(c)([^a-z]*)(k)',
            ' f[!@#\$%\^\&\*]*u[!@#\$%\^&\*]*k', 'f u u c',
            '(f)(c|[^a-z ])(u|[^a-z ])(k)', r'f\*',
            'feck ', ' fux ', 'f\*\*', 
            'f\-ing', 'f\.u\.', 'f###', ' fu ', 'f@ck', 'f u c k', 'f uck', 'f ck'
        ],
    ' ass ':
        [
            '[^a-z]ass ', '[^a-z]azz ', 'arrse', ' arse ', '@\$\$'
                                                           '[^a-z]anus', ' a\*s\*s', '[^a-z]ass[^a-z ]',
            'a[@#\$%\^&\*][@#\$%\^&\*]', '[^a-z]anal ', 'a s s'
        ],
    ' ass hole ':
        [
            ' a[s|z]*wipe', 'a[s|z]*[w]*h[o|0]+[l]*e', '@\$\$hole'
        ],
    ' bitch ':
        [
            'b[w]*i[t]*ch', 'b!tch',
            'bi\+ch', 'b!\+ch', '(b)([^a-z]*)(i)([^a-z]*)(t)([^a-z]*)(c)([^a-z]*)(h)',
            'biatch', 'bi\*\*h', 'bytch', 'b i t c h'
        ],
    ' bastard ':
        [
            'ba[s|z]+t[e|a]+rd'
        ],
    ' trans gender':
        [
            'transgender'
        ],
    ' gay ':
        [
            'gay'
        ],
    ' cock ':
        [
            '[^a-z]cock', 'c0ck', '[^a-z]cok ', 'c0k', '[^a-z]cok[^aeiou]', ' cawk',
            '(c)([^a-z ])(o)([^a-z ]*)(c)([^a-z ]*)(k)', 'c o c k'
        ],
    ' dick ':
        [
            ' dick[^aeiou]', 'deek', 'd i c k'
        ],
    ' suck ':
        [
            'sucker', '(s)([^a-z ]*)(u)([^a-z ]*)(c)([^a-z ]*)(k)', 'sucks', '5uck', 's u c k'
        ],
    ' cunt ':
        [
            'cunt', 'c u n t'
        ],
    ' bull shit ':
        [
            'bullsh\*t', 'bull\$hit'
        ],
    ' homo sex ual':
        [
            'homosexual'
        ],
    ' jerk ':
        [
            'jerk'
        ],
    ' idiot ':
        [
            'i[d]+io[t]+', '(i)([^a-z ]*)(d)([^a-z ]*)(i)([^a-z ]*)(o)([^a-z ]*)(t)', 'idiots'
                                                                                      'i d i o t'
        ],
    ' dumb ':
        [
            '(d)([^a-z ]*)(u)([^a-z ]*)(m)([^a-z ]*)(b)'
        ],
    ' shit ':
        [
            'shitty', '(s)([^a-z ]*)(h)([^a-z ]*)(i)([^a-z ]*)(t)', 'shite', '\$hit', 's h i t'
        ],
    ' shit hole ':
        [
            'shythole'
        ],
    ' retard ':
        [
            'returd', 'retad', 'retard', 'wiktard', 'wikitud'
        ],
    ' rape ':
        [
            ' raped'
        ],
    ' dumb ass':
        [
            'dumbass', 'dubass'
        ],
    ' ass head':
        [
            'butthead'
        ],
    ' sex ':
        [
            'sexy', 's3x', 'sexuality'
        ],
    ' nigger ':
        [
            'nigger', 'ni[g]+a', ' nigr ', 'negrito', 'niguh', 'n3gr', 'n i g g e r'
        ],
    ' shut the fuck up':
        [
            'stfu'
        ],
    ' pussy ':
        [
            'pussy[^c]', 'pusy', 'pussi[^l]', 'pusses'
        ],
    ' faggot ':
        [
            'faggot', ' fa[g]+[s]*[^a-z ]', 'fagot', 'f a g g o t', 'faggit',
            '(f)([^a-z ]*)(a)([^a-z ]*)([g]+)([^a-z ]*)(o)([^a-z ]*)(t)', 'fau[g]+ot', 'fae[g]+ot',
        ],
    ' mother fucker':
        [
            ' motha ', ' motha f', ' mother f', 'motherucker',
        ],
    ' whore ':
        [
            'wh\*\*\*', 'w h o r e'
        ],
}


class PatternTokenizer(BaseTokenizer):
    def __init__(self, lower=True, initial_filters=r"[^a-z0-9!@#\$%\^\&\*_\-,\.' ]", patterns=RE_PATTERNS,
                 remove_repetitions=True):
        self.lower = lower
        self.patterns = patterns
        self.initial_filters = initial_filters
        self.remove_repetitions = remove_repetitions

    def process_text(self, text):
        x = self._preprocess(text)
        for target, patterns in self.patterns.items():
            for pat in patterns:
                x = re.sub(pat, target, x)
        x = re.sub(r"[^a-z' ]", ' ', x)
        return x.split()

    def process_ds(self, ds):
        ### ds = Data series

        # lower
        ds = copy.deepcopy(ds)
        if self.lower:
            ds = ds.str.lower()
        # remove special chars
        if self.initial_filters is not None:
            ds = ds.str.replace(self.initial_filters, ' ')
        # fuuuuck => fuck
        if self.remove_repetitions:
            pattern = re.compile(r"(.)\1{2,}", re.DOTALL) 
            ds = ds.str.replace(pattern, r"\1")

        for target, patterns in self.patterns.items():
            for pat in patterns:
                ds = ds.str.replace(pat, target)

        ds = ds.str.replace(r"[^a-z' ]", ' ')

        return ds.str.split()

    def _preprocess(self, text):
        # lower
        if self.lower:
            text = text.lower()

        # remove special chars
        if self.initial_filters is not None:
            text = re.sub(self.initial_filters, ' ', text)

        # fuuuuck => fuck
        if self.remove_repetitions:
            pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
            text = pattern.sub(r"\1", text)
        return text
    

def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string

    return text

In [ ]:
class Data:
    def __init__(self):
        df = pd.read_csv("../input/jigsaw-dataset/all_in_one_jigsaw.csv")
        df = self.get_text_label_df(df)
        self.df = self.clean(df)
        
    def get_text_label_df(self, df):
        df['y'] = (df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) > 0 ).astype(int)
        df = df[['comment_text_processed', 'y']].rename(columns={'comment_text_processed': 'text'})
        df = df.dropna()
        return df
    
    def unsample(self, df, random_state=402):
        min_len = (df['y'] == 1).sum()
        df_y0_undersample = df[df['y'] == 0].sample(n=min_len, random_state=random_state)
        df = pd.concat([df[df['y'] == 1], df_y0_undersample])
        return df
    
    def clean(self, df):
        tqdm.pandas()
        df['text'] = df['text'].progress_apply(text_cleaning)
        return df
    
    def to_dataset(self):
        ds = Dataset.from_pandas(self.df)
        ds.save_to_disk("./jigsaw_dataset")

In [ ]:
data = Data()

In [ ]:
data.to_dataset()

In [ ]:
!ls ./jigsaw_dataset/ -lh

```python
tokenizer = PatternTokenizer()
final["comment_text_processed"] = tokenizer.process_ds(final["comment_text"]).str.join(sep=" ")

tqdm.pandas()
df['text'] = df['text'].progress_apply(text_cleaning)
```

In [ ]:
# %%time
# df = pd.read_csv("../input/jigsaw-dataset/all_in_one_jigsaw.csv")
# df['y'] = (df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) > 0 ).astype(int)
# df = df[['comment_text_processed', 'y']].rename(columns={'comment_text_processed': 'text'})
# df = df.dropna()
# df.shape

- comment_text_processed is the text which will be used in training.
- ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'] for label
- we will use the same data preprocess in the [notebook](https://www.kaggle.com/adldotori/all-in-one-dataset/notebook)

<h3>Text Cleaning</h3>

In [ ]:
# tqdm.pandas()
# df_val['less_toxic'] = df_val['less_toxic'].progress_apply(text_cleaning)
# df_val['more_toxic'] = df_val['more_toxic'].progress_apply(text_cleaning)

# Undersampling

In [ ]:
# df['y'].value_counts()
# df['y'].value_counts(normalize=True)

# min_len = (df['y'] == 1).sum()
# df_y0_undersample = df[df['y'] == 0].sample(n=min_len, random_state=201)
# df = pd.concat([df[df['y'] == 1], df_y0_undersample])

# TF-IDF

In [ ]:
# vec = TfidfVectorizer(min_df= 3, max_df=0.5, analyzer = 'char_wb', ngram_range = (3,5))
# X = vec.fit_transform(df['text'])

# model = LogisticRegression()
# model.fit(X, df['y'])

# df_val = pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")

<h2>Text cleaning</h2>

In [ ]:
# X_less_toxic = vec.transform(df_val['less_toxic'])
# X_more_toxic = vec.transform(df_val['more_toxic'])

# p1 = model.predict_proba(X_less_toxic)
# p2 = model.predict_proba(X_more_toxic)

# # Validation Accuracy
# (p1[:, 1] < p2[:, 1]).mean()

# Prepare submission data 

In [ ]:
# df_sub = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
# tqdm.pandas()
# df_sub['text'] = df_sub['text'].progress_apply(text_cleaning)
# X_test = vec.transform(df_sub['text'])
# p3 = model.predict_proba(X_test)
# df_sub['score'] = p3[:, 1]
# df_sub['score'].count()
# df_sub['score'] = df_sub['score'] 
# # 9 comments will fail if compared one with the other
# df_sub['score'].nunique()

In [ ]:
# df_sub[['comment_id', 'score']].to_csv("submission.csv", index=False)